## ⚙️ Package Installation & Version Check

**IMPORTANT:** Cloud-based red teaming requires `azure-ai-projects` version `1.1.0b3` or later.

### Check Your Current Version

Run this cell first to check if you have the correct version installed:

In [ ]:
# Check and install/upgrade to the required version
import importlib.util
import subprocess
import sys

def check_and_install():
    needs_install = []
    
    # Check azure-ai-projects version
    try:
        import azure.ai.projects
        if azure.ai.projects.__version__ != "1.1.0b3":
            needs_install.append("azure-ai-projects==1.1.0b3")
    except ImportError:
        needs_install.append("azure-ai-projects==1.1.0b3")
    
    # Check azure-identity
    if importlib.util.find_spec("azure.identity") is None:
        needs_install.append("azure-identity")
    
    # Check python-dotenv
    if importlib.util.find_spec("dotenv") is None:
        needs_install.append("python-dotenv")
    
    if needs_install:
        print(f"📦 Installing/upgrading: {', '.join(needs_install)}")
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q", 
            "--disable-pip-version-check", "--upgrade"
        ] + needs_install)
        print("\n✅ Installation complete! Please restart the kernel and run this notebook from the beginning.")
    else:
        print("✅ All required packages are already installed at the correct versions!")

check_and_install()

### Install/Upgrade Required Packages

If the check above shows you need to upgrade, run this cell:

**Note:** This will upgrade your azure-ai-projects package to the beta version required for cloud red teaming.

In [1]:
# Check installed version
try:
    import azure.ai.projects
    print(f"✅ azure-ai-projects version: {azure.ai.projects.__version__}")
    
    # Try to import RedTeam to verify cloud red teaming support
    try:
        from azure.ai.projects.models import RedTeam
        print("✅ Cloud red teaming features available!")
    except ImportError:
        print("❌ Cloud red teaming features NOT available!")
        print("   Your version is too old. You need version 1.1.0b3 or later.")
        print("\n   Please run the installation cell below.")
except ImportError:
    print("❌ azure-ai-projects not installed!")
    print("   Please run the installation cell below.")

✅ azure-ai-projects version: 1.1.0b4
✅ Cloud red teaming features available!


# Azure AI Red Teaming - Cloud-Based Scan

**Lab 03: Running Red Team Scans Against Azure OpenAI Models**

In this lab, you'll learn how to use Azure AI Foundry's built-in red teaming capabilities to test your AI models for vulnerabilities and harmful content generation.

## What You'll Learn

1. How to connect to Azure AI Foundry Project from code
2. How to configure and launch a red team scan
3. How to monitor scan progress and retrieve results
4. How to interpret findings in the Azure AI Foundry Portal

## Prerequisites

Before running this notebook, ensure you have:
- ✅ Run `source ../../infra/2-setup-env.sh` to configure environment variables
- ✅ Logged in with `az login`
- ✅ An Azure OpenAI model deployed in your project

---

## 1. Import Required Libraries

First, we'll import all the necessary Python packages:
- **os, sys, Path**: For file and environment management
- **dotenv**: To load environment variables from `.env` file
- **azure.identity**: For Azure authentication
- **azure.ai.projects**: To interact with Azure AI Foundry
- **Azure AI Projects models**: To configure cloud-based red team scans with specific attack strategies

**Important:** For cloud-based red teaming, we use `azure.ai.projects.models` (not `azure.ai.evaluation.red_team`).

In [2]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    RedTeam,
    AzureOpenAIModelConfiguration,
    AttackStrategy,
    RiskCategory,
)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


### 🔍 Troubleshooting Imports

If you encounter import errors, verify your package versions:

```python
import azure.ai.projects
print(f"azure-ai-projects version: {azure.ai.projects.__version__}")
```

**Common issues:**
- **ModuleNotFoundError**: Run `pip install azure-ai-projects==1.1.0b3 azure-identity`
- **ImportError for AttackStrategy/RiskCategory**: Ensure you're importing from `azure.ai.projects.models` (not `azure.ai.evaluation.red_team`)
- The cloud-based red teaming uses different imports than local red teaming (Lab 01)

---

## 2. Load Environment Configuration

We need to load environment variables that contain:
- Azure subscription and resource group information
- Azure AI Project endpoint
- Azure OpenAI model configuration (endpoint, API key, deployment name)

The `.env` file should be in the repository root and populated by running the setup script.

In [3]:
def load_environment():
    """Load environment variables from .env file in repository root."""
    # Navigate to repository root (two levels up from this notebook)
    env_path = Path.cwd().parent.parent / ".env"
    
    if not env_path.exists():
        print(f"❌ Error: .env file not found at: {env_path}")
        print("\n⚠️  Please run: source ../../infra/2-setup-env.sh")
        return False
    
    load_dotenv(env_path)
    print(f"✅ Loaded environment from: {env_path}")
    return True

# Load the environment
if not load_environment():
    raise Exception("Failed to load environment configuration")

✅ Loaded environment from: /workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/.env


## 3. Validate Required Configuration

Before proceeding, we'll verify that all required environment variables are set. This prevents errors later in the process.

**Required Variables:**
- `AZURE_AI_PROJECT_ENDPOINT`: Your AI Foundry project endpoint
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI service endpoint
- `AZURE_OPENAI_API_KEY`: API key for authentication
- `AZURE_AI_DEPLOYMENT_NAME`: Name of your deployed model

In [4]:
def validate_environment():
    """Validate that all required environment variables are set."""
    required_vars = {
        "AZURE_AI_PROJECT_ENDPOINT": "Azure AI Project endpoint URL",
        "AZURE_OPENAI_ENDPOINT": "Azure OpenAI endpoint URL",
        "AZURE_OPENAI_API_KEY": "Azure OpenAI API key",
        "AZURE_AI_DEPLOYMENT_NAME": "Model deployment name",
    }
    
    missing_vars = []
    for var, description in required_vars.items():
        value = os.getenv(var)
        if not value:
            missing_vars.append(f"  ❌ {var}: {description}")
        else:
            # Show partial value for verification (hide sensitive data)
            if "KEY" in var or "SECRET" in var:
                display_value = value[:8] + "..." if len(value) > 8 else "***"
            else:
                display_value = value
            print(f"  ✅ {var}: {display_value}")
    
    if missing_vars:
        print("\n❌ Error: Missing required environment variables:\n")
        print("\n".join(missing_vars))
        print("\n⚠️  Please run: source ../../infra/2-setup-env.sh")
        return False
    
    print("\n✅ All required environment variables are set!")
    return True

# Validate the environment
if not validate_environment():
    raise Exception("Missing required environment variables")

  ✅ AZURE_AI_PROJECT_ENDPOINT: https://aoai-g4o6rvcduotd6.services.ai.azure.com/api/projects/proj-g4o6rvcduotd6
  ✅ AZURE_OPENAI_ENDPOINT: https://aoai-g4o6rvcduotd6.openai.azure.com/
  ✅ AZURE_OPENAI_API_KEY: 85qxLpuB...
  ✅ AZURE_AI_DEPLOYMENT_NAME: gpt-4.1-mini

✅ All required environment variables are set!


## 4. Configure Azure AI Project Connection

Now we'll extract the Azure AI Project endpoint from our environment. This endpoint is used to connect to your specific project in Azure AI Foundry.

The endpoint format is: `https://<hub-name>.services.ai.azure.com/api/projects/<project-name>`

In [5]:
# Get the AI Project endpoint
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
project_name = os.getenv("AZURE_AI_PROJECT")

print("📍 Azure AI Project Configuration:")
print(f"   Project Name: {project_name}")
print(f"   Project Endpoint: {project_endpoint}")
print("\n✅ Project connection configured!")

📍 Azure AI Project Configuration:
   Project Name: proj-g4o6rvcduotd6
   Project Endpoint: https://aoai-g4o6rvcduotd6.services.ai.azure.com/api/projects/proj-g4o6rvcduotd6

✅ Project connection configured!


## 5. Configure Azure OpenAI Model Target

Next, we'll configure the Azure OpenAI model that will be tested by the red team scan.

**What you need:**
- **Model Endpoint**: The URL of your Azure OpenAI service
- **API Key**: Authentication key for the service
- **Deployment Name**: The specific model deployment to test (e.g., gpt-4, gpt-35-turbo)

In [6]:
# Get Azure OpenAI configuration
model_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_deployment_name = os.getenv("AZURE_AI_DEPLOYMENT_NAME")

print("🤖 Azure OpenAI Model Configuration:")
print(f"   Model Deployment: {model_deployment_name}")
print(f"   Model Endpoint: {model_endpoint}")
print(f"   API Key: {model_api_key[:8]}..." if model_api_key else "   API Key: Not set")
print("\n✅ Model target configured!")

🤖 Azure OpenAI Model Configuration:
   Model Deployment: gpt-4.1-mini
   Model Endpoint: https://aoai-g4o6rvcduotd6.openai.azure.com/
   API Key: 85qxLpuB...

✅ Model target configured!


## 6. Initialize Azure AI Project Client

We'll create a client connection to Azure AI Foundry using:
- The project endpoint we configured
- Azure credentials from your `az login` session

This client will be used to submit and manage red team scans.

In [7]:
# Create the AI Project client with Azure credentials
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False)
)

print("✅ Azure AI Project client initialized!")
print("   Using DefaultAzureCredential (from az login)")

✅ Azure AI Project client initialized!
   Using DefaultAzureCredential (from az login)


## 7. Configure the Red Team Scan

Now we'll configure the red team scan with specific parameters:

### Attack Strategies
- **BASE64**: Tests if the model can be tricked by base64-encoded malicious prompts

### Risk Categories
- **VIOLENCE**: Tests if the model generates violent or harmful content

### Target Configuration
- Specifies which model deployment to test

You can customize these parameters to test different attack vectors and risk categories.

In [8]:
# Create target configuration for the Azure OpenAI model
target_config = AzureOpenAIModelConfiguration(
    model_deployment_name=model_deployment_name
)

# Configure the Red Team Agent
red_team_agent = RedTeam(
    attack_strategies=[AttackStrategy.BASE64],  # Test base64 encoding obfuscation
    risk_categories=[RiskCategory.VIOLENCE],     # Test for violent content
    display_name="red-team-cloud-lab03",         # Unique name for this scan
    target=target_config,
)

print("⚙️  Red Team Scan Configuration:")
print("   ─────────────────────────────")
print("   Display Name: red-team-cloud-lab03")
print("   Target Model:", model_deployment_name)
print("   Attack Strategies: BASE64")
print("   Risk Categories: VIOLENCE")
print("\n✅ Scan configuration ready!")

⚙️  Red Team Scan Configuration:
   ─────────────────────────────
   Display Name: red-team-cloud-lab03
   Target Model: gpt-4.1-mini
   Attack Strategies: BASE64
   Risk Categories: VIOLENCE

✅ Scan configuration ready!


## 8. Submit the Red Team Scan

Now we'll submit the red team scan to Azure AI Foundry. The scan will:
1. Generate adversarial prompts based on the attack strategies
2. Send these prompts to your Azure OpenAI model
3. Analyze the model's responses for vulnerabilities
4. Generate a detailed report

**Note:** The scan runs asynchronously and may take several minutes to complete.

In [9]:
# Create headers with model credentials
# This allows the red team service to call your Azure OpenAI model
headers = {
    "model-endpoint": model_endpoint,
    "api-key": model_api_key
}

print("🚀 Submitting red team scan...")
print("   This may take a moment...\n")

try:
    # Submit the scan
    red_team_response = project_client.red_teams.create(
        red_team=red_team_agent, 
        headers=headers
    )
    
    print("✅ Scan created successfully!")
    print("   ─────────────────────────")
    print(f"   Scan Name: {red_team_response.name}")
    print(f"   Status: {red_team_response.status}")
    
    # Store scan name for later use
    scan_name = red_team_response.name
    
except Exception as e:
    print(f"\n❌ Error submitting scan: {str(e)}")
    print("\n💡 Troubleshooting tips:")
    print("   1. Verify you're logged in: az login")
    print("   2. Check your .env file has correct values")
    print("   3. Ensure the model deployment exists in your project")
    print("   4. Verify you have permissions to create red team scans")
    raise

🚀 Submitting red team scan...
   This may take a moment...

✅ Scan created successfully!
   ─────────────────────────
   Scan Name: 0171e619-828c-4ddb-b832-05c2846c6a63
   Status: NotStarted


## 9. Check Scan Status

Let's check the current status of our red team scan. The scan can be in one of several states:
- **Pending**: Scan is queued and waiting to start
- **Running**: Scan is actively testing the model
- **Completed**: Scan finished successfully
- **Failed**: Scan encountered an error

You can run this cell multiple times to monitor progress.

In [10]:
# Get the current status of the scan
get_red_team_response = project_client.red_teams.get(name=scan_name)

print("📊 Red Team Scan Status:")
print("   ────────────────────")
print(f"   Scan Name: {scan_name}")
print(f"   Current Status: {get_red_team_response.status}")

if get_red_team_response.status == "Running":
    print("\n   ⏳ Scan is running...")
    print("      This may take several minutes to complete.")
    print("      Re-run this cell to check for updates.")
elif get_red_team_response.status == "Completed":
    print("\n   ✅ Scan completed!")
    print("      Results are ready to view in the portal.")
elif get_red_team_response.status == "Failed":
    print("\n   ❌ Scan failed.")
    print("      Check the portal for error details.")
else:
    print(f"\n   Status: {get_red_team_response.status}")

📊 Red Team Scan Status:
   ────────────────────
   Scan Name: 0171e619-828c-4ddb-b832-05c2846c6a63
   Current Status: Starting

   Status: Starting


## 10. List All Red Team Scans

Let's view all red team scans in this project. This helps you:
- Track multiple scans over time
- Compare results from different configurations
- Monitor the history of your security testing

In [11]:
print("📋 All Red Team Scans in This Project:")
print("   " + "="*60 + "\n")

scans = list(project_client.red_teams.list())

if not scans:
    print("   No scans found.")
else:
    for idx, scan in enumerate(scans, 1):
        print(f"   {idx}. Scan Name: {scan.name}")
        print(f"      Status: {scan.status}")
        if hasattr(scan, 'created_at') and scan.created_at:
            print(f"      Created: {scan.created_at}")
        if hasattr(scan, 'display_name') and scan.display_name:
            print(f"      Display Name: {scan.display_name}")
        print()

print(f"\n   Total scans: {len(scans)}")

📋 All Red Team Scans in This Project:

   1. Scan Name: 4f211bf9-dc21-4a2c-9610-4a8a707ab2a0
      Status: Completed
      Display Name: Lab2d-Custom-Prompt-Scan

   2. Scan Name: 08942307-1aca-4e3b-a11e-923430118180
      Status: Completed
      Display Name: Agent-Scan

   3. Scan Name: 54e705c3-bfff-4164-a2f9-1e9c2f5bd54e
      Status: Completed
      Display Name: Lab2c-Advanced-AppCallback

   4. Scan Name: 48ed2dae-5761-40ed-b6b7-d07101eed370
      Status: Completed
      Display Name: Lab2b-Intermediate-ModelTarget

   5. Scan Name: 13f5711c-23c2-4929-b38f-0ecaf138e767
      Status: Completed
      Display Name: Lab2a-Basic-FixedCallback


   Total scans: 5


## 11. View Results in Azure AI Foundry Portal

The Azure AI Foundry Portal provides a rich interface for viewing red team scan results, including:
- **Attack Logs**: See each adversarial prompt that was tested
- **Response Analysis**: View how your model responded to each attack
- **Risk Scores**: Understand the severity of identified vulnerabilities
- **Mitigation Recommendations**: Get actionable guidance on improving your model

Run this cell to get the direct link to your scan results.

In [12]:
# Extract project information for portal URL
endpoint_parts = project_endpoint.split('/')
project_name_from_endpoint = endpoint_parts[-1]
hub_domain = endpoint_parts[2].split('.')[0]

# Construct portal URL
portal_url = f"https://ai.azure.com/project/{project_name_from_endpoint}/redteam"

print("🌐 View Results in Azure AI Foundry Portal")
print("   " + "="*70 + "\n")
print(f"   🔗 Portal URL: {portal_url}\n")
print("   📖 Steps to View Results:")
print("   ─────────────────────────")
print("   1. Click the portal URL above (or copy and paste to browser)")
print(f"   2. Navigate to your project: {project_name}")
print("   3. Click on 'Red Team' in the left navigation panel")
print("   4. Find your scan: red-team-cloud-lab03")
print("   5. Click on the scan to view detailed results")
print("\n   📊 What You'll Find:")
print("   ───────────────────")
print("   • Adversarial prompts that were tested")
print("   • Model responses and how they were scored")
print("   • Risk assessment and severity ratings")
print("   • Recommendations for improving model safety")
print("   • Detailed attack logs and analysis")

print("\n✅ Red team scan process complete!")

🌐 View Results in Azure AI Foundry Portal

   🔗 Portal URL: https://ai.azure.com/project/proj-g4o6rvcduotd6/redteam

   📖 Steps to View Results:
   ─────────────────────────
   1. Click the portal URL above (or copy and paste to browser)
   2. Navigate to your project: proj-g4o6rvcduotd6
   3. Click on 'Red Team' in the left navigation panel
   4. Find your scan: red-team-cloud-lab03
   5. Click on the scan to view detailed results

   📊 What You'll Find:
   ───────────────────
   • Adversarial prompts that were tested
   • Model responses and how they were scored
   • Risk assessment and severity ratings
   • Recommendations for improving model safety
   • Detailed attack logs and analysis

✅ Red team scan process complete!


## 12. Summary and Next Steps

🎉 **Congratulations!** You've successfully:
- ✅ Connected to Azure AI Foundry from code
- ✅ Configured a red team scan with specific attack strategies
- ✅ Submitted the scan to test your Azure OpenAI model
- ✅ Monitored the scan status
- ✅ Learned how to view results in the portal

### 🔍 Next Steps

1. **Review the Results**: Visit the Azure AI Foundry Portal to analyze your scan findings
2. **Try Different Configurations**: Modify the attack strategies and risk categories to test other scenarios
3. **Test Multiple Models**: Compare how different models respond to the same attacks
4. **Implement Mitigations**: Apply the recommendations from the scan results
5. **Continuous Testing**: Make red teaming a regular part of your AI development workflow

### 📚 Additional Attack Strategies to Try

- `AttackStrategy.JAILBREAK`: Tests prompt injection vulnerabilities
- `AttackStrategy.XPIA`: Tests cross-domain prompt injection attacks

### 📚 Additional Risk Categories to Try

- `RiskCategory.HATE_UNFAIRNESS`: Tests for biased or hateful content
- `RiskCategory.SEXUAL`: Tests for inappropriate sexual content
- `RiskCategory.SELF_HARM`: Tests for self-harm related content

---

**Need Help?** Check the [Azure AI Foundry Documentation](https://learn.microsoft.com/azure/ai-studio/)